Install Dependencies

In [ ]:
! apt-get update
! pip install --upgrade pip

! apt-get install linux-headers-$(uname -r)
! apt-get -y install cmake

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,992 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [1,021 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [982 kB]
Get:1

In [ ]:
!gcc --version

In [ ]:
!nvcc --version

In [ ]:
! pip install ipython
! pip install scipy
! pip install h5py

In [ ]:
! pip install ninja yacs cython matplotlib tqdm opencv-python overrides tensorboard

In [ ]:
! pip install torch==1.10.0+cu113 torchvision==0.11.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch
torch.cuda.is_available()
torch.__version__

install cocoapi

In [ ]:
! git clone https://github.com/cocodataset/cocoapi.git
! cd cocoapi/PythonAPI; python setup.py build_ext install

install apex

In [ ]:
! wget --quiet "https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py"
! python collect_env.py

In [ ]:
# install cuda 11.3 if version is wrong
# since nvidia is updating its gpg key, use the new keyring package and remove duplicate .list files

! sudo apt-key del 7fa2af80
! wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-keyring_1.0-1_all.deb
! sudo dpkg -i cuda-keyring_1.0-1_all.deb
! mv /etc/apt/sources.list.d/cuda.list /etc/apt/sources.list.d/cuda.list.bak
! apt-get update
! apt-get install cuda-11.3

In [ ]:
! git clone https://github.com/AndersonALAC/apex.git
! cd apex ; python setup.py install --cuda_ext --cpp_ext

install scene graph benchmark

In [ ]:
! rm -rf scene_graph

In [ ]:
! git clone https://github.com/AndersonALAC/scene_graph.git

In [ ]:
! cd scene_graph; git pull origin master
! cd scene_graph; git checkout master

In [ ]:
! cd scene_graph; python setup.py build develop

link datasets

In [ ]:
# ! wget https://cs.stanford.edu/people/rak248/VG_100K_2/images.zip
# ! wget https://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip

! cp /content/drive/MyDrive/datasets/vg/images.zip /content/images.zip
! cp /content/drive/MyDrive/datasets/vg/images2.zip /content/images2.zip

In [ ]:
! unzip /content/images.zip
! unzip /content/images2.zip
! mv -v /content/VG_100K_2/* /content/VG_100K/
! mv /content/VG_100K/ /content/scene_graph/datasets/vg/VG_100K/
! rm /content/VG_100K_2/

In [ ]:
# soft link to save checkpoints to drive
! mkdir /content/scene_graph/datasets/vg/detector_model/
! cp -R /content/drive/MyDrive/checkpoints/pretrained_faster_rcnn/ /content/scene_graph/datasets/vg/detector_model/pretrained_faster_rcnn/
! cp /content/drive/MyDrive/datasets/vg/VG-SGG-with-attri.h5 /content/scene_graph/datasets/vg/VG-SGG-with-attri.h5
! ln -s /content/drive/MyDrive/checkpoints /content/checkpoints

run model training

In [ ]:
! cd scene_graph; git pull origin master
! cd scene_graph; git checkout master

In [ ]:
 # sha-dlfe
#  ! cd scene_graph; CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --master_port $RANDOM --nproc_per_node=1 tools/relation_train_net.py --config-file "configs/SHA_GCL_e2e_relation_X_101_32_8_FPN_1x.yaml" GLOBAL_SETTING.DATASET_CHOICE 'VG' GLOBAL_SETTING.RELATION_PREDICTOR 'TransLikePredictor' GLOBAL_SETTING.BASIC_ENCODER 'Hybrid-Attention' GLOBAL_SETTING.GCL_SETTING.GROUP_SPLIT_MODE 'divide4' GLOBAL_SETTING.GCL_SETTING.KNOWLEDGE_TRANSFER_MODE 'KL_logit_TopDown' MODEL.ROI_RELATION_HEAD.USE_GT_BOX False MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False SOLVER.IMS_PER_BATCH 8 TEST.IMS_PER_BATCH 1 DTYPE "float16" SOLVER.MAX_ITER 24000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 GLOVE_DIR /content/glove OUTPUT_DIR /content/checkpoints/SHA_DLFE_SGDet_2 MODEL.BALANCED_NORM True

In [ ]:
# motif
# ! cd scene_graph; CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --master_port $RANDOM --nproc_per_node=1 tools/relation_train_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" MODEL.ROI_RELATION_HEAD.USE_GT_BOX False MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False MODEL.ROI_RELATION_HEAD.PREDICTOR MotifPredictor SOLVER.IMS_PER_BATCH 8 TEST.IMS_PER_BATCH 1 DTYPE "float16" SOLVER.MAX_ITER 24000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 GLOVE_DIR /content/glove MODEL.PRETRAINED_DETECTOR_CKPT /content/checkpoints/pretrained_faster_rcnn/model_final.pth OUTPUT_DIR /content/checkpoints/Motif_SGDet_2

In [ ]:
# vctree
! cd scene_graph; CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --master_port 10025 --nproc_per_node=1 tools/relation_train_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" MODEL.ROI_RELATION_HEAD.USE_GT_BOX False MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False MODEL.ROI_RELATION_HEAD.PREDICTOR VCTreePredictor SOLVER.IMS_PER_BATCH 8 TEST.IMS_PER_BATCH 1 DTYPE "float16" SOLVER.MAX_ITER 24000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 GLOVE_DIR /content/glove MODEL.PRETRAINED_DETECTOR_CKPT /content/checkpoints/pretrained_faster_rcnn/model_final.pth OUTPUT_DIR /content/checkpoints/VCTREE_SGDet_2